<a href="https://colab.research.google.com/github/Aman-Gautam1/Chat_bot-with-Rag-and-Mysql-Storage/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fetching wikipedia content using Langchain

In [ ]:
!pip install langchain wikipedia

In [ ]:
!pip install langchain-community

In [ ]:
!pip install chromadb sentence-transformers

In [ ]:
!pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 3.1 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders import WikipediaLoader
loader = WikipediaLoader(query='History of India')
docs = loader.load()


#chunk and preprocess the text using `RECUSRSIVETEXTSPLITTER`



In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 250,
    chunk_overlap  = 70
)

In [ ]:
clean_text = docs[0].page_content.strip()
clean_text = " ".join(clean_text.split())
chunks = splitter.split_documents(docs)
print(f"total chunks are {len(chunks)}")


total chunks are 606


# Embedding and vector store

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
model1 = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_db = Chroma.from_documents(chunks, model1)

<ipython-input-19-b20e885ec6a7>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  model1 = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


# open source  model

In [ ]:
from langchain_groq  import ChatGroq
api_key = 'your api key'

model = ChatGroq(model ='deepseek-r1-distill-llama-70b',groq_api_key=api_key)

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
prompt =ChatPromptTemplate.from_messages(
    [   '''
        You are an AI agent who Answer the questions based on the provided context only
        Please provide the most accurate response from the context
        <context>
        {context}
        </context>

        question: {input}
        Provide a direct answer without any additional explanation or thinking process.
        '''
    ]
)

#Creating chains for optimal output

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
document_chain = create_stuff_documents_chain(model,prompt)
retriever = vector_db.as_retriever()
retrieval_chain = create_retrieval_chain(retriever,document_chain)

In [ ]:
def querry_output(retrieval_chain,query):
  response = retrieval_chain.invoke({'input':query})
  return response['answer']

In [ ]:
query = 'Powerful '
ans = querry_output(retrieval_chain,query)
def output(response):
  if '</think>' in response:
    # Extract the content after the </think> tag
    start_idx = response.find('</think>') + len('</think>')
    answer = response[start_idx:].strip()  # Get the text after </think>
  return answer


In [ ]:
output(ans)


'The powerful kingdoms of India mentioned in the context are the Vijayanagara Empire, Rajput states, the Bahmani Sultanate (which split into five Deccan sultanates), and the Bengal Sultanate. \n\nAnswer: Vijayanagara Empire, Rajput states, Bahmani Sultanate, Bengal Sultanate.'